In [43]:
import pandas as pd
import numpy as np
import itertools as it
from datetime import datetime
import networkx as nx
# Documentacion de la libreria: http://networkx.readthedocs.io/en/networkx-1.11/

from operator import itemgetter
from itertools import groupby

In [44]:
def filterProblems(df, isTraining, date):
    """
        Funcion que devuelve el conjunto de problemas que tienen status AC o PE
        Si isTraining es true, entonces la funcion sacara el training_set, si no, sacara el evaluation_set
        date es la fecha de particion
    """
    
    if isTraining:
        df = df[df['submissionDate'] < date]
        df = df.loc[df['status'].isin(['AC', 'PE'])]
    else:
        df = df[df['submissionDate'] >= date]
    
    

    return df


In [45]:
def compareNodes(f_list, s_list):
    """
        Funcion que devuelve el numero de usuarios que han hecho ambos problemas
    """
    peso = len(np.intersect1d(f_list, s_list))
    
    return peso
    
def createLinks(prob_us_set, nodos):
    """
        Funcion que crea los enlaces del grafo a partir de la informacion contenida en el conjunto que se le
        pasa a la funcion
    """
    resultado = list() 
    
    # hago todas las posibles combinaciones de problemas
    for fst, snd in it.combinations(nodos, 2):
        # obtengo el peso pasando la lista de usuarios que ha hecho cada problema
        peso = compareNodes(prob_us_set[fst], prob_us_set[snd])
        if peso >= 1:
            resultado.append((fst, snd, peso))
            
            
            
    return resultado

In [46]:
def filterWeight(weightUmbral, linksToFilter):
    """
        Funcion que filtra los enlaces de un grafo, para que el peso sea mayor o igual al dado
    """
    
    result = [(x, y, z) for (x, y, z) in linksToFilter if z >= weightUmbral]
    
    return result
    

In [47]:
def create_graph_nx(list_nodes, list_links):
    """
        Funcion que crea un grafo de tipo Graph de la libreria NetworkX
        Construccion del grafo: http://networkx.readthedocs.io/en/networkx-1.11/tutorial/tutorial.html#what-to-use-as-nodes-and-edges
    """
    grafo = nx.Graph() # creo la variable grafo

    # incluyo los nodos del grafo 
    grafo.add_nodes_from(list_nodes)

    # se incluyen las tuplas de enlaces con el peso del enlace
    # es una lista de la forma [(Nodo1, Nodo2, peso), ......]
    grafo.add_weighted_edges_from(list_links)

    return grafo

In [48]:
# MAIN
# ---------

# se guarda en la variable df (DataFrame) toda la base de datos
df = pd.read_csv('bbdd_orderbydate.csv')

# aqui quito los problemas que no existian despues de la fecha umbral
df = df[df['problem_id'] <= 511] 

# construyo el conjunto de entrenamiento
training_set = filterProblems(df, True, "2015-07-01 00:00:00")

print(len(training_set))

# obtengo los nodos del grafo:
nodes = training_set.problem_id.unique()


# creo un diccionario que va a tener a los problemas como keys y los valores seran los
# usuarios que han hecho ese problema
grouped = training_set.groupby('problem_id')['user_id'].apply(list)




4712


In [49]:
print(training_set)

      problem_id  user_id status       submissionDate
0             10        5     AC  2014-02-17 15:27:07
1              2        6     AC  2014-02-17 15:39:17
2              2        9     AC  2014-02-18 00:30:14
3             10        9     AC  2014-02-18 00:34:46
4              4        9     AC  2014-02-18 00:50:28
5              6        9     AC  2014-02-18 00:52:11
6             13        9     AC  2014-02-18 00:53:40
9             15        8     AC  2014-02-19 19:58:03
10             4        8     AC  2014-02-20 14:23:30
11            39       16     AC  2014-02-20 15:44:33
12            39       17     AC  2014-02-20 16:53:34
13            13       12     AC  2014-02-21 11:08:38
15            33       12     AC  2014-02-21 11:58:32
16            39       12     AC  2014-02-21 12:05:49
17            44       12     AC  2014-02-21 12:52:44
18            44       15     AC  2014-02-21 14:51:52
19            44       22     AC  2014-02-21 14:58:55
21            19        8   

In [50]:
# OBTENCION DEL EVALUATION_SET
# -------

# ahota saco el evaluation_set
evaluation_set = filterProblems(df, False, "2015-07-01 00:00:00")

print(evaluation_set)

# creo un diccionario que va a tener a los usuarios como keys y a los problemas que ha hecho como valores
# a partir del conjunto de entrenamiento
grouped_user_eval = evaluation_set.groupby('user_id')['problem_id'].apply(list)

# convierto la serie en un dataframe
df_users_eval = pd.DataFrame({'user_id':grouped_user_eval.index, 'list_problem_id':grouped_user_eval.values})

print(df_users_eval)

       problem_id  user_id status       submissionDate
6933          117      725     WA  2015-07-01 09:17:38
6934          122      725    RTE  2015-07-01 11:09:50
6935            2     1665     CE  2015-07-01 13:15:06
6936          340     1504     WA  2015-07-01 16:12:24
6937          443     1735     TL  2015-07-01 18:55:30
6938           39     1737     AC  2015-07-01 20:36:18
6939          134     1317     AC  2015-07-02 13:37:18
6940          150     1504     AC  2015-07-02 19:43:11
6941          183     1504     AC  2015-07-02 20:37:12
6942           39     1736     AC  2015-07-03 00:00:53
6943          195     1504     AC  2015-07-03 03:53:16
6944          508     1504     AC  2015-07-03 04:19:22
6945            2     1504     AC  2015-07-03 04:29:37
6946          174     1504     AC  2015-07-03 17:16:06
6947           35      103     AC  2015-07-03 18:10:52
6948          181     1504     AC  2015-07-03 22:08:51
6949            8     1736     AC  2015-07-04 15:46:08
6950      

In [51]:

# creo los enlaces a partir de la informacion de los nodos
links = createLinks(grouped, nodes)
# ahora filtro el grafo para que los enlaces solo tengan el peso que quiero
linksFiltered = filterWeight(5, links)

print(len(linksFiltered))

# aqui creo el grafo 
graph = create_graph_nx(nodes, linksFiltered)

4443


In [52]:
print(len(links))
print(len(nodes))

13493
169


In [53]:
def apply_cn(row, graph):
    """
        Funcion que devuelve el numero de vecinos en comun de esos dos nodos
    """
    return len(list(nx.common_neighbors(graph, row['one'], row['two'])))

def create_cn_data(graph, nodes):

    # Ahora voy a construir un DataFrame que tenga dos columnas con todas las posibles combinaciones de problemas, y otra 
    # columna con el valor de cn para ese par de problemas
    fst_column = list()
    snd_column = list()
    for fst, snd in it.combinations(nodes, 2):
        fst_column.append(fst)
        snd_column.append(snd)

    d = {'one' : fst_column,
        'two' : snd_column}
    dataFrame_cn = pd.DataFrame(d)


    # Aplico la funcion a cada fila
    dataFrame_cn['cn'] = dataFrame_cn.apply (lambda row: apply_cn(row, graph), axis=1)


    return dataFrame_cn

In [54]:
# Hasta aqui se hace la creacion del grafo
cn_df = create_cn_data(graph, nodes)
print(cn_df)

       one  two  cn
0       10    2  48
1       10    4  43
2       10    6  38
3       10   13  42
4       10   15  46
5       10   39  48
6       10   33  44
7       10   44  45
8       10   19  30
9       10   27  24
10      10   60  23
11      10  100  45
12      10   93   0
13      10   53  41
14      10   51  41
15      10   49  47
16      10   70  44
17      10   62  47
18      10   35  36
19      10   81   5
20      10    8  33
21      10   17   8
22      10   23  23
23      10   25   0
24      10   86  15
25      10   29  44
26      10   31   0
27      10   47   0
28      10   65   0
29      10   73  29
...    ...  ...  ..
14166  510  509  31
14167  510  507   0
14168  511  504   0
14169  511  503  32
14170  511  505  12
14171  511  506   0
14172  511  502  19
14173  511  509  34
14174  511  507   0
14175  504  503   0
14176  504  505   0
14177  504  506   0
14178  504  502   0
14179  504  509   0
14180  504  507   0
14181  503  505  13
14182  503  506   0
14183  503  502  21


In [55]:
# creo un diccionario que va a tener a los usuarios como keys y a los problemas que ha hecho como valores
# a partir del conjunto de entrenamiento
grouped_user = training_set.groupby('user_id')['problem_id'].apply(list)

# convierto la serie en un dataframe
df_users = pd.DataFrame({'user_id':grouped_user.index, 'list_problem_id':grouped_user.values})

print(df_users)

                                       list_problem_id  user_id
0                                        [10, 76, 489]        5
1                                                  [2]        6
2                     [39, 44, 76, 105, 109, 114, 117]        7
3    [15, 4, 19, 27, 60, 53, 105, 76, 83, 90, 147, ...        8
4                  [2, 10, 4, 6, 13, 253, 57, 62, 143]        9
5                                            [33, 109]       11
6    [13, 33, 39, 44, 100, 51, 49, 70, 81, 53, 130,...       12
7                                            [44, 187]       15
8                                                 [39]       16
9                                             [39, 86]       17
10                                                [44]       22
11                                           [100, 13]       24
12                 [258, 141, 309, 310, 390, 187, 510]       25
13                                               [100]       28
14                                      

In [56]:
def lenProblemsDone(row, set_filter):
    """
        Funcion auxiliar que calcula cuanto problemas ha hecho cada usuario en un conjunto: training o evaluation
    """
    # saco el dataframe que contendra solo una fila con la lista de problemas que ha hecho el usuario
    df_filter = set_filter[set_filter['user_id'] == row['user_id']]
    
    if df_filter.empty:
        # si esta vacio, entonces es que el usuario no ha hecho problemas en ese conjunto
        return 0
    else:
        # sino, devuelvo la longitud de la lista de problemas
        return len(df_filter['list_problem_id'].iloc[0]) 
    
    

In [57]:
# aqui voy a hacer el filtro de usuarios de forma que para hacer las recomendaciones solo tengamos en 
# cuenta aquellos usuarios que han hecho 5 o mas problemas tanto antes de la fecha limite como despues

# primero guardo la lista de usuarios
user_list = df.user_id.unique()

# la meto en un dataframe 
column_user_filter = {'user_id': user_list}
datraframe_user_filter = pd.DataFrame.from_dict(column_user_filter)


# ahora tengo que calcular para cada fila, el numero de problemas que han hecho en el training_set, evaluation_set
datraframe_user_filter['len_training'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users), axis=1)
datraframe_user_filter['len_evaluation'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users_eval), axis=1)
print(datraframe_user_filter)


      user_id  len_training  len_evaluation
0           5             3               2
1           6             1               1
2           9             9               5
3           8            40               8
4          16             1               0
5          17             2               2
6          12            12               0
7          18             0              13
8          15             2               2
9          22             1               0
10          7             7               0
11         27             0               0
12         28             1               0
13         29             1               0
14         32             2               0
15         35            40               2
16         33             7               2
17         39            11               0
18         31             9               0
19         40             1               0
20         44             8               0
21         48             0     

In [58]:
# ahora tengo que hacer el filtro en este dataframe, de forma que solo aparezcan las filas en las que len_training y 
# len_evaluation sea >=5
datraframe_user_filter = datraframe_user_filter[(datraframe_user_filter['len_training'] >= 5) & (datraframe_user_filter['len_evaluation'] >=5)]
print(datraframe_user_filter)

# aqui voy a guardar la lista de usuarios a los que voy a recomendar
user_list_to_recommend = sorted(datraframe_user_filter['user_id'].tolist())
print(user_list_to_recommend)
print(len(user_list_to_recommend))

      user_id  len_training  len_evaluation
2           9             9               5
3           8            40               8
29         60             6              27
118        25             7              33
261       130            62              34
274       382             5               6
285       206           111              38
286        62           112              32
299       414             9             103
310       437           134              13
316       443            41              22
317       448            31              33
333       381            13              10
348       103           109              48
383       542             8              13
395       431            11               8
483       689             5              41
597       254            11              12
601       864            14               8
619       912             7              17
621       840            26              19
633       935            88     

In [59]:
# ahora tengo que filtrar df_users para que solo contenga las filas en las que los usuarios
# pertenecen a la anterior lista

df_users = df_users[df_users['user_id'].isin(user_list_to_recommend)]
print(df_users)

                                       list_problem_id  user_id
3    [15, 4, 19, 27, 60, 53, 105, 76, 83, 90, 147, ...        8
4                  [2, 10, 4, 6, 13, 253, 57, 62, 143]        9
12                 [258, 141, 309, 310, 390, 187, 510]       25
37                            [100, 62, 2, 57, 53, 23]       60
38   [178, 251, 224, 159, 191, 183, 166, 139, 256, ...       62
69   [33, 51, 60, 70, 83, 134, 181, 187, 191, 206, ...      103
71        [10, 191, 228, 441, 437, 438, 436, 109, 122]      105
88   [309, 150, 183, 237, 191, 187, 39, 209, 70, 15...      130
135  [53, 119, 251, 256, 224, 253, 181, 155, 191, 1...      206
162  [259, 141, 250, 254, 70, 255, 195, 159, 252, 3...      254
224  [259, 235, 226, 51, 119, 316, 258, 352, 272, 2...      381
225                          [119, 262, 264, 114, 388]      382
237         [213, 2, 109, 114, 10, 436, 438, 437, 404]      414
245  [39, 259, 258, 203, 251, 60, 49, 195, 6, 159, 70]      431
248  [250, 124, 249, 100, 4, 122, 57, 30

In [60]:
# con esto creo un dataframe que separa el dataframe anterior
# lista de usuario - problema en el conjunto de entrenamiento
df_new = df_users.groupby('user_id').list_problem_id.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)

df_new.columns = ['user_id','problem']

print(df_new)

      user_id  problem
0           8       15
1           8        4
2           8       19
3           8       27
4           8       60
5           8       53
6           8      105
7           8       76
8           8       83
9           8       90
10          8      147
11          8      241
12          8      245
13          8      249
14          8      206
15          8      195
16          8      203
17          8      209
18          8      213
19          8      217
20          8      226
21          8      228
22          8      222
23          8       86
24          8      279
25          8        2
26          8        6
27          8      250
28          8      251
29          8      253
...       ...      ...
1066     1485      309
1067     1485      327
1068     1485      314
1069     1485      141
1070     1485      259
1071     1485      469
1072     1485      441
1073     1485      352
1074     1485      325
1075     1485      346
1076     1485      155
1077     14

In [61]:
def getKrecommendations(row, cn_df, df_users, k):
    """
        Funcion que devuelve la lista de k mejores problemas para el usuario dado teniendo en cuenta que 
        las recomendaciones no son problemas que ya haya realziado el usuario
    """
    #column_result = cn_df[cn_df['one'] == 10].sort_values('cn', ascending=False)
    # user_list = df_users[df_users['user_id'] == 6]
    
   
    
    # primero saco los dos dataframes con problemas que se pueden recomendar
    column_result_one = cn_df[cn_df['one'] == row['problem']]
    column_result_two = cn_df[cn_df['two'] == row['problem']]
    
    
    tmp1 = column_result_two['two'].tolist()
    tmp2 = column_result_two['one'].tolist()
    tmp3 = column_result_two['cn'].tolist()
   
    # creo un nuevo df
    df_tmp = pd.DataFrame({'one':tmp1, 'two':tmp2, 'cn': tmp3})
    
    frames = [column_result_one, df_tmp]
    
    # concateno los resultados
    column_result_tmp = pd.concat(frames)
    
    # ordeno los problemas que se pueden recomendar
    column_result_tmp2 = column_result_tmp.sort_values('cn', ascending=False)
     
    tmp1 = column_result_tmp2['one'].tolist()
    tmp2 = column_result_tmp2['two'].tolist()
    tmp3 = column_result_tmp2['cn'].tolist()
    
    # creo un nuevo df     
    column_result = pd.DataFrame({'one':tmp1, 'two':tmp2, 'cn': tmp3})
    
    # con esto consigo sacar la lista de problemas que ha realizado ese usuario
    user_list = df_users[df_users['user_id'] == row['user_id']]
    user_list = user_list['list_problem_id'].iloc[0] 
    
    # ahora filtro la columna para que los problemas recomendados no los haya hecho ya el usuario
    column_result = column_result[column_result['two'].isin(user_list) == False]
    #print((column_result['two'].head(k)).tolist())
    return (column_result['two'].head(k)).tolist()
    

In [62]:
# Aplico la funcion a cada fila 
k = 1
df_new['list_recommendations'] = df_new.apply (lambda row: getKrecommendations(row, cn_df,  df_users, k), axis=1)
print(df_new)


      user_id  problem list_recommendations
0           8       15                 [39]
1           8        4                [109]
2           8       19                [254]
3           8       27                [134]
4           8       60                [183]
5           8       53                 [39]
6           8      105                [109]
7           8       76                [468]
8           8       83                [109]
9           8       90                [109]
10          8      147                 [49]
11          8      241                [109]
12          8      245                [254]
13          8      249                 [62]
14          8      206                 [49]
15          8      195                 [39]
16          8      203                [109]
17          8      209                 [39]
18          8      213                 [39]
19          8      217                [134]
20          8      226                 [39]
21          8      228          

In [63]:
# creo un nuevo dataframe que agrupa por el primer problema 
df_separation = df_new.groupby(['user_id', 'problem']).list_recommendations.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_2', axis = 1)

df_separation.columns = ['user_id','problem', 'recommendation']

print(df_separation)

      user_id  problem  recommendation
0           8        2           109.0
1           8        4           109.0
2           8        6           109.0
3           8       10            39.0
4           8       13            39.0
5           8       15            39.0
6           8       19           254.0
7           8       27           134.0
8           8       53            39.0
9           8       60           183.0
10          8       76           468.0
11          8       83           109.0
12          8       86            29.0
13          8       90           109.0
14          8      105           109.0
15          8      147            49.0
16          8      195            39.0
17          8      203           109.0
18          8      206            49.0
19          8      209            39.0
20          8      213            39.0
21          8      217           134.0
22          8      222           313.0
23          8      226            39.0
24          8      228   

In [64]:
def asignaCNvalue(row, cn_df):
    """
        Funcion que devuelve una columna con los valores de cn asociados a cada fila
    """
    
    # print(cn_df.one == row['problem'])
    # print(cn_df.two)
    # print(row['recommendation'])
    # print(cn_df.two == row['recommendation'])
    # print((cn_df.one == row['problem']) & (cn_df.two == row['recommendation']))
    
    new_df = cn_df[(cn_df.one == row['problem']) & (cn_df.two == row['recommendation'])]
    
    if new_df.empty:
        new_df = cn_df[(cn_df.two == row['problem']) & (cn_df.one == row['recommendation'])]
    
    return new_df.iloc[0]['cn']

In [65]:
df_separation['cn'] = df_separation.apply(lambda row: asignaCNvalue(row, cn_df), axis=1)
print(df_separation)

      user_id  problem  recommendation   cn
0           8        2           109.0  104
1           8        4           109.0   88
2           8        6           109.0   64
3           8       10            39.0   48
4           8       13            39.0   90
5           8       15            39.0   99
6           8       19           254.0   45
7           8       27           134.0   34
8           8       53            39.0   88
9           8       60           183.0   28
10          8       76           468.0    0
11          8       83           109.0   71
12          8       86            29.0   17
13          8       90           109.0   46
14          8      105           109.0  102
15          8      147            49.0   49
16          8      195            39.0   81
17          8      203           109.0  108
18          8      206            49.0   45
19          8      209            39.0  102
20          8      213            39.0   50
21          8      217          

In [66]:
# ahora lo que quiero es ordenar los problemas por cada usuario en funcion de su cn
# primero ordeno por su valor de user y luego por el de cn, de forma que quedan ordenador por su valor cn
df_separation = df_separation.sort_values(by=['user_id', 'cn'], ascending=False)
print(df_separation)

      user_id  problem  recommendation   cn
1089     1619       39           109.0  122
1092     1619      100            62.0  108
1090     1619       44           109.0  102
1086     1619       15           109.0   99
1095     1619      443           254.0   89
1094     1619      257           254.0   74
1085     1619        6           109.0   64
1093     1619      252           256.0   57
1087     1619       19           254.0   45
1088     1619       27           134.0   34
1091     1619       60           183.0   28
1083     1504      109            39.0  122
1084     1504      187            39.0  102
1078     1504       15            39.0   99
1081     1504       29            39.0   94
1082     1504       35            39.0   68
1079     1504       19           254.0   45
1080     1504       23            39.0   34
1050     1485       39           109.0  122
1061     1485      259           254.0  109
1051     1485       49           109.0  108
1053     1485      134          

In [67]:
# ahora tengo que hacer un nuevo dataframe con usuario, problema, y una lista de recommendation 
# (los tres primeros ya que estan ordenados por cn)

# hago primero la agrupacion por usuario
grouped_r = df_separation.groupby('user_id')

# hago la agregacion en una lista 
df_recommend = grouped_r.aggregate(lambda x:list(x))

print(df_recommend)

                                                   problem  \
user_id                                                      
8        [259, 256, 258, 203, 2, 241, 105, 209, 15, 13,...   
9                      [62, 2, 13, 4, 253, 6, 143, 10, 57]   
25                     [258, 309, 187, 310, 390, 141, 510]   
60                                [62, 100, 2, 53, 57, 23]   
62       [39, 109, 254, 258, 49, 62, 259, 134, 256, 44,...   
103      [39, 109, 254, 203, 49, 62, 2, 134, 256, 258, ...   
105           [109, 191, 436, 441, 438, 10, 228, 437, 122]   
130      [39, 109, 62, 134, 309, 100, 203, 2, 388, 44, ...   
206      [39, 2, 62, 109, 49, 134, 254, 258, 259, 33, 4...   
254      [254, 259, 390, 70, 141, 159, 195, 255, 349, 2...   
381      [109, 259, 258, 209, 316, 471, 235, 352, 51, 2...   
382                              [388, 119, 114, 262, 264]   
414             [109, 2, 436, 404, 438, 213, 10, 114, 437]   
431      [39, 258, 259, 49, 203, 70, 159, 195, 251, 6, 60]   
437     

In [68]:
# para sacar el dataframe final con user - krecomendaciones
del df_recommend['problem']
del df_recommend['cn']

In [69]:
def delRepetitions(row):
    """
        Funcion auxiliar para evitar que salgan repeticiones en las recomendaciones. Saco la lista de posibles 
        recomendaciones con valores unicos
    """
    conjunto_vacio = set()
    
    # esto sirve para que se haga mas rapido la comprobacion de si el elemento esta en la lista o no
    function_add = conjunto_vacio.add
    
    # hago la lista intensional, para mantener el orden dado en la lista original
    return [x for x in row['recommendation'] if not (x in conjunto_vacio or function_add(x))]

In [70]:
# ahora voy a aplicar una funcion a la lista de posibles recomendaciones, para quitar los repetidos
df_recommend['recommendation'] = df_recommend.apply(lambda row: delRepetitions(row), axis=1)
print(df_recommend)

                                            recommendation
user_id                                                   
8        [39.0, 254.0, 109.0, 49.0, 134.0, 62.0, 183.0,...
9                                [39.0, 109.0, 49.0, 53.0]
25                             [254.0, 39.0, 307.0, 134.0]
60                                     [39.0, 109.0, 29.0]
62       [2.0, 312.0, 308.0, 114.0, 8.0, 95.0, 76.0, 28...
103      [209.0, 105.0, 141.0, 251.0, 159.0, 136.0, 235...
105                        [39.0, 254.0, 316.0, 2.0, 62.0]
130      [254.0, 256.0, 258.0, 307.0, 49.0, 259.0, 241....
206      [51.0, 162.0, 114.0, 308.0, 178.0, 8.0, 90.0, ...
254              [109.0, 39.0, 307.0, 256.0, 388.0, 209.0]
381                                   [39.0, 254.0, 233.0]
382                                   [39.0, 109.0, 105.0]
414                     [39.0, 254.0, 258.0, 316.0, 191.0]
431                           [109.0, 254.0, 256.0, 183.0]
437      [44.0, 390.0, 70.0, 33.0, 253.0, 441.0, 13.0, .

In [71]:
def getKrecomFinal(row, k):
    """
        Funcion que saca las k mejores recomendaciones para el usuario
        Lo que hace es coger los primeros k valores de la lista de recomendaciones
    """
    if k == 1:
        value = list()
        value.append(row['recommendation'][:k])
        return value
    else:
        return row['recommendation'][:k]

In [72]:
# ahora saco los k mejores problemas para cada usuario
df_recommend['recommendation'] = df_recommend.apply(lambda row: getKrecomFinal(row, k), axis=1)

print(df_recommend)

        recommendation
user_id               
8               [39.0]
9               [39.0]
25             [254.0]
60              [39.0]
62               [2.0]
103            [209.0]
105             [39.0]
130            [254.0]
206             [51.0]
254            [109.0]
381             [39.0]
382             [39.0]
414             [39.0]
431            [109.0]
437             [44.0]
443            [259.0]
448            [254.0]
542             [39.0]
686            [109.0]
689             [39.0]
840            [109.0]
847             [39.0]
864             [39.0]
912            [109.0]
935             [39.0]
979            [109.0]
1026           [109.0]
1037           [109.0]
1136           [109.0]
1139            [39.0]
1190           [109.0]
1212           [109.0]
1246            [62.0]
1317            [39.0]
1386            [39.0]
1436           [109.0]
1456           [109.0]
1460           [109.0]
1480           [109.0]
1485           [109.0]
1504            [39.0]
1619       

In [73]:
# hago el filtro para los usuarios a los que tengo que recomendar
df_users_eval = df_users_eval[df_users_eval['user_id'].isin(user_list_to_recommend)]
print(df_users_eval)

                                       list_problem_id  user_id
2              [445, 436, 437, 441, 442, 443, 444, 62]        8
3                             [83, 109, 136, 141, 254]        9
9    [438, 13, 228, 143, 251, 502, 503, 508, 509, 4...       25
16   [134, 15, 147, 233, 247, 254, 309, 139, 150, 3...       60
17   [2, 60, 247, 17, 128, 114, 308, 86, 65, 130, 9...       62
21   [35, 507, 119, 76, 81, 93, 95, 97, 105, 117, 1...      103
22   [15, 44, 62, 233, 316, 114, 247, 39, 134, 159,...      105
24   [471, 486, 503, 509, 25, 86, 316, 325, 327, 33...      130
30   [247, 311, 17, 128, 114, 308, 86, 65, 130, 340...      206
34   [438, 443, 33, 256, 203, 6, 309, 2, 342, 436, ...      254
43    [510, 441, 438, 325, 237, 183, 315, 327, 358, 2]      381
44                              [33, 8, 465, 6, 2, 13]      382
46   [239, 62, 187, 206, 313, 316, 504, 465, 191, 2...      414
50             [253, 33, 252, 443, 444, 441, 404, 503]      431
51   [489, 181, 390, 393, 397, 437, 486,

In [74]:
# primero voy a ordenar la lista de usuarios a recomendar
user_list_to_recommend.sort()

list_eval_problems = df_users_eval['list_problem_id'].tolist()
list_recom_problems = df_recommend['recommendation'].tolist()


# meto toda la informacion en un dataframe para obtener las metricas
set_df_metric = {'user_id': user_list_to_recommend, 'eval_problems': list_eval_problems, 'recom_problems': list_recom_problems}
metric_df = pd.DataFrame.from_dict(set_df_metric)

print(metric_df)

                                        eval_problems recom_problems  user_id
0             [445, 436, 437, 441, 442, 443, 444, 62]         [39.0]        8
1                            [83, 109, 136, 141, 254]         [39.0]        9
2   [438, 13, 228, 143, 251, 502, 503, 508, 509, 4...        [254.0]       25
3   [134, 15, 147, 233, 247, 254, 309, 139, 150, 3...         [39.0]       60
4   [2, 60, 247, 17, 128, 114, 308, 86, 65, 130, 9...          [2.0]       62
5   [35, 507, 119, 76, 81, 93, 95, 97, 105, 117, 1...        [209.0]      103
6   [15, 44, 62, 233, 316, 114, 247, 39, 134, 159,...         [39.0]      105
7   [471, 486, 503, 509, 25, 86, 316, 325, 327, 33...        [254.0]      130
8   [247, 311, 17, 128, 114, 308, 86, 65, 130, 340...         [51.0]      206
9   [438, 443, 33, 256, 203, 6, 309, 2, 342, 436, ...        [109.0]      254
10   [510, 441, 438, 325, 237, 183, 315, 327, 358, 2]         [39.0]      381
11                             [33, 8, 465, 6, 2, 13]         [3

In [75]:
def one_hit(row):
    """
        Funcion que implementa la metrica one hit. Devuelve un 1 si para un usuarios dado, al menos uno
        de los problemas que se le ha recomendado ha sido realizado por ese usuario en el evaluation_set. 
        Cero si no hay ningun problema de los recomendados que haya sido realizado por el usuario
    """
    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    if len(num_problems_common) >= 1:
        return 1
    else:
        return 0

In [76]:
def mrr(row): 
    """
        Funcion que va a implementar la metrica de evaluacion mrr:
        mrr = 1/ranki, donde ranki es la posicion del primer item correcto
    """

    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    if len(num_problems_common) >= 1:

        # hago la busqueda del primer elemento que esta en la lista de recomendados
        fst_correct_item = -1
        encontrado = False
        i = 0
        while (i < len(row['recom_problems'])) and (encontrado == False):
            if row['recom_problems'][i] in row['eval_problems']:
                # fst_correct_item = row['recom_problems'][i]
                # print(fst_correct_item)
                ranki = i + 1
                encontrado = True
            else:
                i = i + 1
                
        return (1/ranki)

    else:
        return 0


In [77]:
def precision(row):
    """
        Funcion que va a implementar la metrica precision en k: 
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los recomendados
    """
    
    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row['recom_problems']))

In [78]:
def recall(row):
    """
        Funcion que implementa la metrica recall
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los evaluados
    """
    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row['eval_problems']))

In [79]:
def f1(row):
    """
        Funcion que calcula el f1 en funcion de precision y recall
    """
    denominador = row['precision'] + row['recall']
    
    if denominador == 0:
        return 0
    else:
        return (2 * row['precision'] * row['recall']) / denominador

In [80]:
# ahora voy a calcular una metrica para cada usuario
metric_df['one_hit'] = metric_df.apply(lambda row: one_hit(row), axis=1)
metric_df['mrr'] = metric_df.apply(lambda row: mrr(row), axis=1)
metric_df['precision'] = metric_df.apply(lambda row: precision(row), axis=1)
metric_df['recall'] = metric_df.apply(lambda row: recall(row), axis=1)
metric_df['f1'] = metric_df.apply(lambda row: f1(row), axis=1)
print(metric_df)

                                        eval_problems recom_problems  user_id  \
0             [445, 436, 437, 441, 442, 443, 444, 62]         [39.0]        8   
1                            [83, 109, 136, 141, 254]         [39.0]        9   
2   [438, 13, 228, 143, 251, 502, 503, 508, 509, 4...        [254.0]       25   
3   [134, 15, 147, 233, 247, 254, 309, 139, 150, 3...         [39.0]       60   
4   [2, 60, 247, 17, 128, 114, 308, 86, 65, 130, 9...          [2.0]       62   
5   [35, 507, 119, 76, 81, 93, 95, 97, 105, 117, 1...        [209.0]      103   
6   [15, 44, 62, 233, 316, 114, 247, 39, 134, 159,...         [39.0]      105   
7   [471, 486, 503, 509, 25, 86, 316, 325, 327, 33...        [254.0]      130   
8   [247, 311, 17, 128, 114, 308, 86, 65, 130, 340...         [51.0]      206   
9   [438, 443, 33, 256, 203, 6, 309, 2, 342, 436, ...        [109.0]      254   
10   [510, 441, 438, 325, 237, 183, 315, 327, 358, 2]         [39.0]      381   
11                          

In [81]:
# Para crear un archivo grafo para GEPHI


# creo los enlaces a partir de la informacion de los nodos
# links = createLinks(grouped, nodes)


# aqui creo el grafo 
# graph = create_graph_nx(nodes, links)
# nx.write_gexf(graph,"grafo.gexf")

In [82]:
# calculo la media de las metricas

result_one_hit = metric_df['one_hit'].mean()
result_precision = metric_df['precision'].mean()
result_mrr = metric_df['mrr'].mean()
result_recall = metric_df['recall'].mean()
result_f1 = metric_df['f1'].mean()

print("One hit ----------")
print(result_one_hit)
print("Precision ----------")
print(result_precision)
print("Mrr  ----------")
print(result_mrr)
print("Recall  ----------")
print(result_recall)
print("F1  ----------")
print(result_f1)

One hit ----------
0.47619047619047616
Precision ----------
0.47619047619047616
Mrr  ----------
0.47619047619047616
Recall  ----------
0.02334695408790915
F1  ----------
0.043353937504476275


In [83]:

f = open("C:/hlocal/TFM/nodos_problemas", 'a')
f.write(str(result_one_hit) + '\t' + str(result_precision) + '\t' + str(result_mrr) + '\t' + str(result_recall) + '\t' +  str(result_f1) + '\n') 
f.close()